In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


In [13]:
#Reading data file WildFires 1878 - 2018
wildfires_1_df = pd.read_csv("California_Fire_Perimeters.csv")
wildfires_1_df

,OBJECTID,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM
0,1,2007.0,CA,CCO,LAC,OCTOBER,00246393,2007/10/21 00:00:00+00,2007/10/23 00:00:00+00,14.0,NaN,NaN,25.736713,8.0,1.0,00233414
1,2,2007.0,CA,CCO,LAC,MAGIC,00233077,2007/10/22 00:00:00+00,2007/10/25 00:00:00+00,14.0,NaN,NaN,2824.877197,8.0,1.0,00233077
2,3,2007.0,CA,USF,ANF,RANCH,00000166,2007/10/20 00:00:00+00,2007/11/15 00:00:00+00,2.0,NaN,54716.0,58410.335938,7.0,1.0,00000166
3,4,2007.0,CA,CCO,LAC,EMMA,00201384,2007/09/11 00:00:00+00,2007/09/11 00:00:00+00,14.0,NaN,NaN,172.214951,8.0,1.0,00201384
4,5,2007.0,CA,CCO,LAC,CORRAL,00259483,2007/11/24 00:00:00+00,2007/11/27 00:00:00+00,14.0,NaN,NaN,4707.997070,8.0,1.0,00259483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20503,21123,2018.0,CA,CDF,SLU,RESERVOIR,00010073,2018/09/16 00:00:00+00,2018/09/16 00:00:00+00,11.0,Electrical Power,53.0,51.217384,7.0,1.0,NaN
20504,21124,2018.0,CA,CDF,SLU,COLONY,00010007,2018/09/14 00:00:00+00,2018/09/14 00:00:00+00,11.0,Electrical Power,2.0,2.110911,7.0,1.0,NaN
20505,21125,2018.0,CA,CDF,SLU,CAMINO,00011247,2018/10/19 00:00:00+00,2018/10/19 00:00:00+00,10.0,No damage to structures or infrastructure,58.0,58.527103,7.0,1.0,NaN
20506,21126,2018.0,CA,BIA,HIA,SIGNBOARD,00000174,2018/11/16 00:00:00+00,2018/11/17 00:00:00+00,NaN,NaN,NaN,81.154282,3.0,1.0,NaN


In [3]:
wildfires_df["STATE"].value_counts()

CA    20416
NV       73
OR       15
AZ        1
Name: STATE, dtype: int64

In [9]:
year_max = wildfires_1_df["YEAR_"].max()
year_min = wildfires_1_df["YEAR_"].min()

print(f"Year Max: {year_max}")
print(f"Year Min: {year_min}")

Year Max: 2018.0
Year Min: 1878.0


In [14]:
# Reading file for Wild Fiers 2013 - 2020 in California only
wildfires_2_df = pd.read_csv("California_Fire_Incidents.csv")
wildfires_2_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [15]:
# Temperature
# Reading file for Wild Fiers 1743 - 2013 in California only
temp_2 = pd.read_csv("GlobalLandTemperaturesByState.csv")
temp_2.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [22]:
us_only = temp_2[temp_2['Country'] == "United States"]
us_only

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
7458,1743-11-01,10.722,2.898,Alabama,United States
7459,1743-12-01,NaN,NaN,Alabama,United States
7460,1744-01-01,NaN,NaN,Alabama,United States
7461,1744-02-01,NaN,NaN,Alabama,United States
7462,1744-03-01,NaN,NaN,Alabama,United States
...,...,...,...,...,...
626435,2013-05-01,10.607,0.208,Wyoming,United States
626436,2013-06-01,16.267,0.276,Wyoming,United States
626437,2013-07-01,20.222,0.133,Wyoming,United States
626438,2013-08-01,19.621,0.217,Wyoming,United States


In [23]:
us_only['dt'].max()

'2013-09-01'

In [24]:
us_only['dt'].min()

'1743-11-01'

In [28]:
# Population
# Reading file for Population 1743 - 2013 in California only
population_df = pd.read_csv("cal_populations_city.csv")
population_df

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089
...,...,...,...,...,...,...,...
450,Orange,Yorba Linda,1967,28254.0,52422,58918,64234
451,Napa,Yountville,1965,2893.0,3259,3297,2933
452,Siskiyou,Yreka,1857,5916.0,6948,7290,7765
453,Sutter,Yuba City,1908,18736.0,27437,36758,64925
